### 1. Importing libraries and data

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier, KerasRegressor
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [3]:
path = r'C:\Users\nodnarb\Documents\Machine Learning'

In [4]:
# Importing cleaned weather station data

df = df = pd.read_csv(os.path.join(path, '01 Data Sets', 'weather_cleaned.csv'))

In [5]:
# Importing pleasant weather data

pleasant = pd.read_csv(os.path.join(path, '01 Data Sets', 'pleasant_weather.csv'))

### 2. Data Prep

In [7]:
# Dropping date and month column from weather data

df.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [8]:
# Dropping date column from pleasant data

pleasant.drop(columns = 'DATE', inplace = True)

### 3. Data Reshaping and Split

In [10]:
# Turn X and answers from a df to arrays

X = np.array(df)
y = np.array(pleasant)

In [11]:
X = X.reshape(-1,15,9)

In [12]:
X.shape

(22950, 15, 9)

In [13]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [14]:
y.shape

(22950,)

In [15]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [16]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


### 4. Bayesian Optimization

In [19]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [20]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [26]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/25


C:\Users\nodnarb\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nodnarb\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 2s - 160ms/step - accuracy: 0.6040 - loss: 2.7014
Epoch 2/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6895
Epoch 7/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6874
Epoch 8/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6854
Epoch 9/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6836
Epoch 10/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6818
Epoch 11/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6801
Epoch 12/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6785
Epoch 13/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6770
Epoch 14/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6755
Epoch 15/25
15/15 - 0s - 25ms/step - accuracy: 0.6440

C:\Users\nodnarb\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nodnarb\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
15/15 - 2s - 148ms/step - accuracy: 0.6207 - loss: 2.7020
Epoch 2/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6970
Epoch 4/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6853
Epoch 9/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6784
Epoch 13/25
15/15 - 0s - 24ms/step - accuracy: 0.6440 - loss: 2.6768
Epoch 14/25
15/15 - 0s - 23ms/step - accuracy: 0.6440 - loss: 2.6753
Epoch 15/25
15/15 - 0s - 24ms/step - accur

...

C:\Users\nodnarb\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nodnarb\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 - 2s - 71ms/step - accuracy: 0.0089 - loss: 2.8883
Epoch 2/47
30/30 - 0s - 10ms/step - accuracy: 0.0088 - loss: 2.8791
Epoch 3/47
30/30 - 0s - 10ms/step - accuracy: 0.0083 - loss: 2.8718
Epoch 4/47
30/30 - 0s - 10ms/step - accuracy: 0.0099 - loss: 2.8638
Epoch 5/47
30/30 - 0s - 10ms/step - accuracy: 0.0110 - loss: 2.8542
Epoch 6/47
30/30 - 0s - 10ms/step - accuracy: 0.0129 - loss: 2.8458
Epoch 7/47
30/30 - 0s - 10ms/step - accuracy: 0.0127 - loss: 2.8384
Epoch 8/47
30/30 - 0s - 10ms/step - accuracy: 0.0147 - loss: 2.8299
Epoch 9/47
30/30 - 0s - 10ms/step - accuracy: 0.0158 - loss: 2.8187
Epoch 10/47
30/30 - 0s - 10ms/step - accuracy: 0.0184 - loss: 2.8113
Epoch 11/47
30/30 - 0s - 10ms/step - accuracy: 0.0203 - loss: 2.8014
Epoch 12/47
30/30 - 0s - 10ms/step - accuracy: 0.0232 - loss: 2.7897
Epoch 13/47
30/30 - 0s - 10ms/step - accuracy: 0.0249 - loss: 2.7834
Epoch 14/47
30/30 - 0s - 10ms/step - accuracy: 0.0261 - loss: 2.7731
Epoch 15/47
30/30 - 0s - 11ms/step - accuracy: 0.0322 

C:\Users\nodnarb\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\nodnarb\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\nodnarb\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\nodnarb\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nodnarb\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 210, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nodnarb\anaconda3\Lib\site-packages\scikeras\wrappers.py

30/30 - 3s - 84ms/step - accuracy: 0.0585 - loss: 2.8213
Epoch 2/47
30/30 - 0s - 10ms/step - accuracy: 0.0592 - loss: 2.8146
Epoch 3/47
30/30 - 0s - 10ms/step - accuracy: 0.0640 - loss: 2.8065
Epoch 4/47
30/30 - 0s - 10ms/step - accuracy: 0.0650 - loss: 2.7988
Epoch 5/47
30/30 - 0s - 10ms/step - accuracy: 0.0665 - loss: 2.7920
Epoch 6/47
30/30 - 0s - 10ms/step - accuracy: 0.0766 - loss: 2.7811
Epoch 7/47
30/30 - 0s - 10ms/step - accuracy: 0.0794 - loss: 2.7749
Epoch 8/47
30/30 - 0s - 10ms/step - accuracy: 0.0846 - loss: 2.7665
Epoch 9/47
30/30 - 0s - 11ms/step - accuracy: 0.0914 - loss: 2.7577
Epoch 10/47
30/30 - 0s - 11ms/step - accuracy: 0.0954 - loss: 2.7477
Epoch 11/47
30/30 - 0s - 10ms/step - accuracy: 0.1017 - loss: 2.7408
Epoch 12/47
30/30 - 0s - 10ms/step - accuracy: 0.1041 - loss: 2.7317
Epoch 13/47
30/30 - 0s - 10ms/step - accuracy: 0.1197 - loss: 2.7212
Epoch 14/47
30/30 - 0s - 10ms/step - accuracy: 0.1228 - loss: 2.7119
Epoch 15/47
30/30 - 0s - 10ms/step - accuracy: 0.1281 

KeyboardInterrupt: 

In [28]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'tanh',
 'batch_size': 828,
 'dropout': 0.19967378215835974,
 'dropout_rate': 0.15427033152408348,
 'epochs': 38,
 'kernel': 1.0929008254399954,
 'layers1': 2,
 'layers2': 1,
 'learning_rate': 0.07440107705542671,
 'neurons': 95,
 'normalization': 0.9656320330745594,
 'optimizer': <keras.src.optimizers.nadam.Nadam at 0x140acb88e60>}

### 5. Optimized Search Parameters

In [32]:
# Set the model with optimized hyperparameters

epochs = 38
batch_size = 828

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 2
layers2 = 1
activation = 'tanh'
kernel = int(round(1.0929008254399954))  # Rounded kernel size for Conv1D
neurons = 95
normalization = 0.9656320330745594
dropout = 0.19967378215835974
dropout_rate = 0.15427033152408348
optimizer = Adadelta(learning_rate=0.07440107705542671)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\nodnarb\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
# Put the y_test set back into a one-hot configuration

y_train1 = to_categorical(y_train, num_classes=15)

In [50]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train1 shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train1 shape: (17212, 15)


In [52]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [54]:
model.fit(X_train, y_train1, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/38
21/21 - 2s - 112ms/step - accuracy: 0.7835 - loss: 0.6375
Epoch 2/38
21/21 - 1s - 25ms/step - accuracy: 0.7867 - loss: 0.6332
Epoch 3/38
21/21 - 1s - 25ms/step - accuracy: 0.7874 - loss: 0.6290
Epoch 4/38
21/21 - 1s - 26ms/step - accuracy: 0.7887 - loss: 0.6249
Epoch 5/38
21/21 - 1s - 25ms/step - accuracy: 0.7888 - loss: 0.6212
Epoch 6/38
21/21 - 1s - 25ms/step - accuracy: 0.7901 - loss: 0.6171
Epoch 7/38
21/21 - 1s - 25ms/step - accuracy: 0.7924 - loss: 0.6133
Epoch 8/38
21/21 - 1s - 25ms/step - accuracy: 0.7920 - loss: 0.6098
Epoch 9/38
21/21 - 1s - 25ms/step - accuracy: 0.7928 - loss: 0.6060
Epoch 10/38
21/21 - 1s - 26ms/step - accuracy: 0.7935 - loss: 0.6028
Epoch 11/38
21/21 - 1s - 29ms/step - accuracy: 0.7950 - loss: 0.5993
Epoch 12/38
21/21 - 1s - 26ms/step - accuracy: 0.7947 - loss: 0.5957
Epoch 13/38
21/21 - 1s - 26ms/step - accuracy: 0.7969 - loss: 0.5914
Epoch 14/38
21/21 - 1s - 26ms/step - accuracy: 0.7965 - loss: 0.5880
Epoch 15/38
21/21 - 1s - 25ms/step - accur

### 6. Confusion Matrix

In [57]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [67]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [69]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [71]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  HEATHROW  LJUBLJANA  MADRID
True                                                                      
BASEL        3409       185        21      11         4          2      50
BELGRADE      228       807        14       6         2          3      32
BUDAPEST       48        60        71       5         5          3      22
DEBILT         27        13        15      21         3          1       2
DUSSELDORF      9         5         2       3         6          0       4
HEATHROW       21         5         2       2        29          1      22
KASSEL          4         2         3       0         0          1       1
LJUBLJANA      21         5         5       0         0         11      19
MAASTRICHT      6         0         0       0         0          0       3
MADRID         53        33         8       0         3          2     359
MUNCHENB        7         1         0       0         0          0       0
OSLO            2        